In [ ]:
from openai import OpenAI  ########## Prompt  ###################

# Setup
api_key = ""
base_url = "https://gpt.uni-muenster.de/v1"
model = "Llama-3.3-70B"  # model used

client = OpenAI(api_key=api_key, base_url=base_url)

# Define scenario
scenario = """
A customer places an order on an e-commerce website. 
The system checks inventory. 
If the item is in stock, customer is informed to pay for the product.
Customer pays for the order.
The payment gateway confirms the payment transaction. 
Finally, the system sends order confirmation email to the customer.
But if the item is not in stock, order is declined.
Customer is sent order declined message.
Customer then decides upon further action either the customer decides to cancel order and it ends
or customer chooses another e-commerce supplier.
"""

# Create the prompt
prompt = f"""

You are an expert in Subject-Oriented Business Process Modeling using the Parallel Activity Specification Schema (PASS).

Analyze the following scenario step-by-step and produce both the Subject Interaction Diagram (SID) and Subject Behavior Diagrams (SBDs).

\"\"\"{scenario}\"\"\"

### Step 1: Identify Subjects

List all autonomous subjects (actors or systems) involved in the scenario.

### Step 2: Identify Messages (Noun Form Only)

List key messages exchanged between subjects using **noun phrases only** (e.g., “Order”, “Inventory Status” etc.).

## Step 3: Subject Interaction Diagram (SID)

Write the SID as a **numbered list** describing subject-to-subject message flows.

Format:
'[Subject A] -> [Subject B]: [Message]'

### Step 4: Subject Behavior Diagram (SBD)

For each subject from Step 1, describe its internal behavior using PASS state types:

- **StartState**, **DoState**, **SendState**, **ReceiveState**, **EndState**

**Rules**:
- 'SendState' :- include 'To:' and 'Msg:'
- 'ReceiveState' :- include 'From:' and 'Msg:'
- Use 'Branches:' for decisions
  Always increment step numbers correctly. Double-check that Branches: refer to valid steps (e.g., "Step: 10", not "Step: 9" if Step 9 is an EndState).
- Use GotoStep: to return to earlier steps (e.g., sending new order again)
-For retry paths (like choosing another supplier), always use GotoStep: instead of duplicating behaviour"
-Ensure branches reference the correct step numbers

- In decision branches:
  - Each `Step:` must point to a valid numbered step that **actually exists**.
  - If retrying earlier steps, use `GotoStep:` and **refer to original step number** (e.g., GotoStep: 2).
- Do not invent new EndStates or GotoSteps that aren't clearly part of a branch.

Example Branch:

6. DoState: Decide next action
   Branches:
   - Step: 7
     Description: Cancel order
   - Step: 9
     Description: Try another supplier

7. SendState: Send Cancellation
8. EndState: Order Cancelled
9. GotoStep: 2  # Retry order

**Important Note on Step Numbering and 'Next:' References:**

- Each step number must be sequential unless explicitly branching or looping back.
- The 'Next:' field in each state should refer to the **very next logical step in the process**, not skip decision points or intermediate steps.
- If a decision is required next, 'Next:' should point to the corresponding DoState step, not jump directly to an EndState or unrelated step.
- Only use 'Next: EndState' when the flow truly terminates at that step. And in 'Next: EndState', put EndState
  number like 'Next: 8) if 8.EndState
- Avoid skipping over intermediate steps such as decisions or sends; maintain logical process flow.

- After receiving confirmations or important messages, flow should proceed to the next logical decision or processing step, not directly to an EndState unless the process actually terminates.

****Important Notes on EndStates:
Use separate EndStates for different outcomes:
e.g., one for “Successfully Completed”, another for “Cancelled”.
Do not route cancellation, decline, or failed attempts to an EndState meant for successful completions.
If a common EndState is used for multiple paths, name it generically, e.g., “Process Completed” or “End of Process”.
Ensure Next: fields route only logically to the correct EndState matching the path.

Model each subject one by one.

### Step 5: Check Logic and Flow

Ensure:
- Each decision branch leads to an `EndState` or valid `GotoStep:`
- No duplicate messages
- Each ReceiveState handles only mutually exclusive messages

### Example Scenario (Healthcare):
A patient books an appointment using a healthcare app.  
The app sends the appointment request to the hospital system.  
The hospital system checks the doctor’s availability.  
If the doctor is available, it confirms the appointment.  
The app then sends a confirmation message to the patient.
If the doctor is not available, hospital system declines the appointment.
The patient is sent decline message.
The patient then decides upon further action either the patient cancels request and all ends
or the patient chooses another hospital.

### SBD:
####Patient:
1. StartState: Decide to make appointment  
   Description: Patient initiates appointment process

2. SendState: Send Appointment Request to Healthcare App  
   To: Healthcare App  
   Msg: Appointment Request

3. ReceiveState: Receive Reply from Healthcare App  
   Choices:
   - From: Healthcare App  
     Msg: Appointment Confirmation  
     Next: 4  
   - From: Healthcare App  
     Msg: Appointment Declination  
     Next: 5  

4. EndState: Appointment booked

5. DoState: Decide upon further action  
   Branches:
   - Step: 6  
     Description: Cancel appointment request  
   - Step: 8  
     Description: Choose another hospital and try again

6. DoState: Cancel appointment request  
   Description: Cancel

7. EndState: Appointment request cancelled (end of this path)
8. GotoStep: 2  # Retry appointment loop

(Similar steps would follow for Healthcare App and Hospital System…)
Now, do the same for the scenario provided above. Output the **SID first**, followed by **SBDs for each subject**.
Model all subjects from Step 1. Do not skip the last subject. Ensure each subject has its full SBD.
Repeat Steps 4–5 for **every subject** listed in Step 1. 
Do not skip subjects 
Ensure each has a complete behavior diagram from StartState to EndState.
"""

# Make the API call
completion = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model=model,
    temperature=0.5    
)

# Output
print(completion.choices[0].message.content)

In [ ]:
llama_output2 = completion.choices[0].message.content

In [1]:
# saved response from LLM in variable below named llama_output2, 
# otherwise comment it out and use llama_output2 from cell immediately above

llama_output2="""
### Step 1: Identify Subjects

The subjects involved in the scenario are:
1. Customer
2. E-commerce Website (System)
3. Payment Gateway

### Step 2: Identify Messages (Noun Form Only)

Key messages exchanged between subjects:
- Order
- Inventory Status
- Payment Request
- Payment Confirmation
- Order Confirmation
- Order Decline
- Cancellation

### Step 3: Subject Interaction Diagram (SID)

1. Customer -> E-commerce Website: Order
2. E-commerce Website -> E-commerce Website: Inventory Status (internal check)
3. E-commerce Website -> Customer: Payment Request
4. Customer -> Payment Gateway: Payment
5. Payment Gateway -> E-commerce Website: Payment Confirmation
6. E-commerce Website -> Customer: Order Confirmation
7. E-commerce Website -> Customer: Order Decline (if item not in stock)
8. Customer -> E-commerce Website: Cancellation (if order declined)

### Step 4: Subject Behavior Diagram (SBD)

#### Customer:
1. StartState: Decide to place order  
   Description: Customer initiates order process

2. SendState: Send Order to E-commerce Website  
   To: E-commerce Website  
   Msg: Order  
   Next: 3

3. ReceiveState: Receive response from E-commerce Website  
   Choices:
   - From: E-commerce Website  
     Msg: Payment Request  
     Next: 4  
   - From: E-commerce Website  
     Msg: Order Decline  
     Next: 6  

4. SendState: Send Payment to Payment Gateway  
   To: Payment Gateway  
   Msg: Payment  
   Next: 5

5. ReceiveState: Receive Payment Confirmation from Payment Gateway via E-commerce Website  
   From: E-commerce Website  
   Msg: Order Confirmation  
   Next: 7

6. DoState: Decide upon further action after order decline  
   Branches:
   - Step: 8  
     Description: Cancel order  
   - Step: 10  
     Description: Choose another supplier and try again

7. EndState: Order confirmed (end of this path)

8. SendState: Send Cancellation to E-commerce Website  
   To: E-commerce Website  
   Msg: Cancellation  
   Next: 9

9. EndState: Order cancelled (end of this path)

10. GotoStep: 2  # Retry order with another supplier

#### E-commerce Website (System):
1. StartState: Receive Order from Customer  
   Description: System initiates order processing

2. DoState: Check inventory status  
   Description: Internal check  
   Branches:
   - Step: 3  
     Description: Item in stock  
   - Step: 5  
     Description: Item not in stock  

3. SendState: Send Payment Request to Customer  
   To: Customer  
   Msg: Payment Request  
   Next: 4

4. ReceiveState: Receive Payment Confirmation from Payment Gateway  
   From: Payment Gateway  
   Msg: Payment Confirmation  
   Next: 6

5. SendState: Send Order Decline to Customer  
   To: Customer  
   Msg: Order Decline  
   Next: 7

6. SendState: Send Order Confirmation to Customer  
   To: Customer  
   Msg: Order Confirmation  
   Next: 8

7. ReceiveState: Receive Cancellation from Customer (if any)  
   From: Customer  
   Msg: Cancellation  
   Next: 9

8. EndState: Order processed successfully (end of this path)

9. EndState: Order cancelled by customer (end of this path)

#### Payment Gateway:
1. StartState: Receive Payment from Customer  
   Description: Payment processing initiated

2. DoState: Process payment  
   Description: Internal processing  
   Next: 3

3. SendState: Send Payment Confirmation to E-commerce Website  
   To: E-commerce Website  
   Msg: Payment Confirmation  
   Next: 4

4. EndState: Payment processed (end of this path)

### Step 5: Check Logic and Flow

- Each decision branch leads to an EndState or a valid GotoStep.
- No duplicate messages.
- Each ReceiveState handles only mutually exclusive messages.

The logic and flow of the SBDs for each subject ensure that the process is correctly modeled, covering all possible paths from order placement to confirmation or cancellation, including payment processing and inventory checks.
"""

In [7]:
# saved response from LLM in variable below named llama_output3, 
# otherwise comment it out and do llama_output3 = completion.choices[0].message.content

llama_output3="""
### Step 1: Identify Subjects

The subjects involved in the scenario are:
1. Customer
2. E-commerce Website (System)
3. Payment Gateway

### Step 2: Identify Messages (Noun Form Only)

Key messages exchanged between subjects:
- Order
- Inventory Status
- Payment Request
- Payment Confirmation
- Order Confirmation
- Order Decline
- Cancellation

### Step 3: Subject Interaction Diagram (SID)

1. Customer -> E-commerce Website: Order
2. E-commerce Website -> E-commerce Website: Inventory Status (internal check)
3. E-commerce Website -> Customer: Payment Request
4. Customer -> Payment Gateway: Payment
5. Payment Gateway -> E-commerce Website: Payment Confirmation
6. E-commerce Website -> Customer: Order Confirmation
7. E-commerce Website -> Customer: Order Decline (if item not in stock)
8. Customer -> E-commerce Website: Cancellation (if order declined)

### Step 4: Subject Behavior Diagram (SBD)

#### Customer:
1. StartState: Decide to place order  
   Description: Customer initiates order process

2. SendState: Send Order to E-commerce Website  
   To: E-commerce Website  
   Msg: Order  
   Next: 3

3. ReceiveState: Receive response from E-commerce Website  
   Choices:
   - From: E-commerce Website  
     Msg: Payment Request  
     Next: 4  
   - From: E-commerce Website  
     Msg: Order Decline  
     Next: 6  

4. SendState: Send Payment to Payment Gateway  
   To: Payment Gateway  
   Msg: Payment  
   Next: 5

5. ReceiveState: Receive Payment Confirmation from Payment Gateway via E-commerce Website  
   From: E-commerce Website  
   Msg: Order Confirmation  
   Next: 7

6. DoState: Decide upon further action after order decline  
   Branches:
   - Step: 8  
     Description: Cancel order  
   - Step: 10  
     Description: Choose another supplier and try again

7. EndState: Order confirmed (end of this path)

8. SendState: Send Cancellation to E-commerce Website  
   To: E-commerce Website  
   Msg: Cancellation  
   Next: 9

9. EndState: Order cancelled (end of this path)

10. GotoStep: 2  # Retry order with another supplier


### Step 5: Check Logic and Flow

- Each decision branch leads to an EndState or a valid GotoStep.
- No duplicate messages.
- Each ReceiveState handles only mutually exclusive messages.

The logic and flow of the SBDs for each subject ensure that the process is correctly modeled, covering all possible paths from order placement to confirmation or cancellation, including payment processing and inventory checks.
"""

In [8]:
import re  ################ PARSING ###################

def parse_sid(sid_text):
    pattern = re.compile(r"\d+\.\s*(.+?)\s*->\s*(.+?):\s*(.+)")
    sid = []
    for line in sid_text.splitlines():
        match = pattern.match(line.strip())
        if match:
            sender = match.group(1).strip()
            receiver = match.group(2).strip()
            message = match.group(3).strip()
            sid.append((f"{sender} -> {receiver}", message))
    return sid

def extract_sbd_section(full_text):
    lines = full_text.splitlines()
    sbd_start = None
    for i, line in enumerate(lines):
         if "SBD" in line.upper():
            sbd_start = i + 1  # start after this line
            break
    if sbd_start is None:
        print("No SBD section found!")
        return ""

    # Extract until explanation or EOF
    sbd_lines = []
    for line in lines[sbd_start:]:
        if line.strip().startswith("### Explanation"):
            break
        sbd_lines.append(line)

    return "\n".join(sbd_lines).strip()

def parse_sbd(sbd_text):
    print("=== SBD Text to parse ===")
    print(sbd_text)
    print("=========================")
    lines = sbd_text.splitlines()

    sbd = {}
    current_system = None
    states = {}
    current_state = None
    current_state_num = None
    choices = []
    branches = []
    parsing_choices = False
    parsing_branches = False

   
    actor_header_re = re.compile(r"^\s*####\s*(.+?):\s*$")
    state_header_re = re.compile(r"(\d+)\.\s+\*{0,2}(\w+)\*{0,2}:\s*(.*)")
    goto_re = re.compile(r"(\d+)\.\s+\*{0,2}GotoStep\*{0,2}:\s*(\d+)")
    choice_re = re.compile(r"-\s*From:\s*(.*)")
    branch_step_re = re.compile(r"-\s*Step:\s*(\d+)")
    branch_desc_re = re.compile(r"Description:\s*(.*)")

    for idx, line in enumerate(lines):
        line = line.rstrip()
        if not line:
            continue

        # Detect actor header
        m = actor_header_re.match(line)
        if m:
             # Finalize the current state
            if current_state is not None:
                if choices:
                    current_state["Choices"] = choices
                if branches:
                    current_state["Branches"] = branches
                if current_state["num"] not in states:
                    states[current_state["num"]] = current_state
                else:
                    existing = states[current_state["num"]]
                    existing.update({k: v for k, v in current_state.items() if k not in existing or k in ("Choices", "Branches")})

            # Save states of previous actor
            if current_system is not None:
                print(f"\nFinalized actor: {current_system}")
                print(f"States: {list(states.values())}")
                sbd[current_system] = list(states.values())
                
            
            # Debug
            print(f"Detected actor: {m.group(1)}")

              

            current_system = m.group(1)
            states = {}
            current_state = None
            current_state_num = None
            choices = []
            branches = []
            parsing_choices = False
            parsing_branches = False
            continue

        if current_system is None:
            # Haven't detected an actor header yet
            current_system = "Default"
            states = {}
            

        m = goto_re.match(line)
        if m:
            current_state_num = int(m.group(1))
            next_step = int(m.group(2))
            current_state = {
                "type": "GotoStep",
                "description": f"Goto step {next_step}",
                "num": current_state_num,
                "GotoStep": next_step
            }
            
            continue

        
                
        m = state_header_re.match(line)
        if m:
           
            # Finalize previous state
            if current_state is not None:
                if choices:
                    print(f"Parsed choices for state {current_state['num']}: {choices}")
                    current_state["Choices"] = choices
                    choices = []
                if branches:
                    print(f"Parsed branches for state {current_state['num']}: {branches}")
                    current_state["Branches"] = branches
                    branches = []
                states[current_state["num"]] = current_state

            parsing_choices = False
            parsing_branches = False


            current_state_num = int(m.group(1))
            state_type = m.group(2)
            description = m.group(3).strip()
            current_state = {
                "type": state_type,
                "description": description,
                "num": current_state_num
            }
            
            # Check next line for edge description, but only if it's StartState or DoState with no branches
            
            if state_type in ["StartState", "DoState"]:
                # Only look ahead if it's not followed by "Branches:"
                next_lines = lines[idx+1:idx+3]  # Look ahead one or two lines safely
                has_branches_soon = any(l.strip() == "Branches:" for l in next_lines)
                if not has_branches_soon:
                    for next_line in next_lines:
                        next_line = next_line.strip()
                        if next_line.startswith("Description:"):
                            current_state["OutgoingLabel"] = next_line.split(":", 1)[1].strip()
                            break

            states[current_state_num] = current_state
            parsing_choices = False
            parsing_branches = False
            continue

        if current_state is None:
            continue

        # Handle SendState To: and Msg:
        if current_state["type"] == "SendState":
            if line.strip().startswith("To:"):
                current_state["To"] = line.split(":",1)[1].strip()
                continue
            if line.strip().startswith("Msg:"):
                current_state["Msg"] = line.split(":",1)[1].strip()
                continue

        
        # Detect Choices and Branches
        if line.strip() == "Choices:":
            if current_state is not None and choices:
                print(f"Parsed choices for state {current_state['num']}: {choices}")
                current_state["Choices"] = choices
                choices = []
            parsing_choices = True
            parsing_branches = False
            
            continue

        elif line.strip() == "Branches:":
            # Save previously parsed branches into current_state before resetting
            if current_state is not None and branches:
                print(f"Parsed branches for state {current_state['num']}: {branches}")
                current_state["Branches"] = branches
                branches = []
            parsing_branches = True
            parsing_choices = False
            
            continue
        

        if parsing_choices:
            m = choice_re.match(line.strip())
            
            if m:
                choices.append({"From": m.group(1).strip()})
            elif line.strip().startswith("Msg:") and choices:
                choices[-1]["Msg"] = line.split(":",1)[1].strip()
            elif line.strip().startswith("Next:") and choices:
                choices[-1]["Next"] = int(line.split(":",1)[1].strip())
            continue

        if parsing_branches:
            m = branch_step_re.match(line.strip())
            if m:
                branches.append({"Step": int(m.group(1))})
            m = branch_desc_re.match(line.strip())
            if m and branches:
                branches[-1]["Description"] = m.group(1).strip()
            continue

    # Save last state's choices or branches
    if current_state is not None:
        if choices:
            current_state["Choices"] = choices
        if branches:
            current_state["Branches"] = branches
        if current_state["num"] not in states:
            states[current_state["num"]] = current_state
        else:
            # Merge new info into existing state
            existing = states[current_state["num"]]
            existing.update({k: v for k, v in current_state.items() if k not in existing or k in ("Choices", "Branches")})
        

    # Save last actor's states
    
    # Save last state's choices or branches
    if current_state is not None:
        if choices:
            current_state["Choices"] = choices
        if branches:
            current_state["Branches"] = branches
        if current_state["num"] not in states:
            states[current_state["num"]] = current_state
        else:
            existing = states[current_state["num"]]
            existing.update({k: v for k, v in current_state.items() if k not in existing or k in ("Choices", "Branches")})

    # Finalize the last actor block (very important!)
    if current_system is not None: 
        print(f"\nFinalized actor: {current_system}")
        print(f"States: {list(states.values())}")
        sbd[current_system] = list(states.values())

    return sbd

    
sbd_textt = extract_sbd_section(llama_output3)

parsed_sbd = parse_sbd(sbd_textt)
print("Parsed SBD::", parsed_sbd)

sid_list = parse_sid(llama_output3.strip())
process = {
    "SID": sid_list
}


print("SID Text:\n", process["SID"])

=== SBD Text to parse ===
#### Customer:
1. StartState: Decide to place order  
   Description: Customer initiates order process

2. SendState: Send Order to E-commerce Website  
   To: E-commerce Website  
   Msg: Order  
   Next: 3

3. ReceiveState: Receive response from E-commerce Website  
   Choices:
   - From: E-commerce Website  
     Msg: Payment Request  
     Next: 4  
   - From: E-commerce Website  
     Msg: Order Decline  
     Next: 6  

4. SendState: Send Payment to Payment Gateway  
   To: Payment Gateway  
   Msg: Payment  
   Next: 5

5. ReceiveState: Receive Payment Confirmation from Payment Gateway via E-commerce Website  
   From: E-commerce Website  
   Msg: Order Confirmation  
   Next: 7

6. DoState: Decide upon further action after order decline  
   Branches:
   - Step: 8  
     Description: Cancel order  
   - Step: 10  
     Description: Choose another supplier and try again

7. EndState: Order confirmed (end of this path)

8. SendState: Send Cancellation to

In [5]:
import os

# Add the environment bin directory to PATH
os.environ["PATH"] += os.pathsep + "/home/s/smanan/.conda/envs/myenv/bin"

# Optional: test that dot is now found
!which dot
!dot -V

/home/s/smanan/.conda/envs/myenv/bin/dot
dot - graphviz version 12.2.1 (20250203.1708)


In [3]:
sbd_textt = sbd_textt.split('### Step 5')[0]
print(sbd_textt)

#### Customer:
1. StartState: Decide to place order  
   Description: Customer initiates order process

2. SendState: Send Order to E-commerce Website  
   To: E-commerce Website  
   Msg: Order  
   Next: 3

3. ReceiveState: Receive response from E-commerce Website  
   Choices:
   - From: E-commerce Website  
     Msg: Payment Request  
     Next: 4  
   - From: E-commerce Website  
     Msg: Order Decline  
     Next: 6  

4. SendState: Send Payment to Payment Gateway  
   To: Payment Gateway  
   Msg: Payment  
   Next: 5

5. ReceiveState: Receive Payment Confirmation from Payment Gateway via E-commerce Website  
   From: E-commerce Website  
   Msg: Order Confirmation  
   Next: 7

6. DoState: Decide upon further action after order decline  
   Branches:
   - Step: 8  
     Description: Cancel order  
   - Step: 10  
     Description: Choose another supplier and try again

7. EndState: Order confirmed (end of this path)

8. SendState: Send Cancellation to E-commerce Website  
   T

In [9]:
####################################### SBD Graph #########################################
import re
from graphviz import Digraph


def safe_filename(name, max_len=50):
    # Remove problematic characters for filenames
    name = re.sub(r'[^\w\-_. ]', '', name)
    # Replace spaces with underscores
    name = name.replace(' ', '_')
    # Truncate to max length
    if len(name) > max_len:
        name = name[:max_len]
    return name

dot = Digraph("SBD", format='pdf')
dot.attr(rankdir='LR')

# Remove any trailing summary or non-subject text
sbd_textt = sbd_textt.split('### Step 5')[0]
subject_blocks = re.split(r'^####\s*(.+?):\s*$', sbd_textt, flags=re.MULTILINE)

print("Total blocks:", len(subject_blocks))
print("Blocks:", subject_blocks)

# Filter out leading junk before first subject (if any)
if len(subject_blocks) > 0 and not subject_blocks[0].strip():
    subject_blocks = subject_blocks[1:]

# Clean subject_blocks if odd (malformed input)
if len(subject_blocks) % 2 != 0:
    print("Warning: Subject block count is not even. Possible parsing issue.")
    subject_blocks = subject_blocks[:-1]  # Remove the last partial item

# Pair subjects and their text
subjects = list(zip(subject_blocks[::2], subject_blocks[1::2]))

print("Subject blocks split result:")
for idx, (subj, txt) in enumerate(subjects):
    print(f"Subject {idx}: {repr(subj[:50])}")


state_styles = {
    'StartState': {"shape": "rectangle", "color": "yellow"},
    'EndState':   {"shape": "rectangle", "color": "yellow"},
    'SendState':  {"shape": "rectangle", "color": "green"},
    'ReceiveState': {"shape": "rectangle", "color": "pink"},
    'DoState':    {"shape": "rectangle", "color": "yellow"},
    'Unknown':    {"shape": "rectangle", "color": "lightgrey"},
}

for subject_name, subject_text in subjects:
    
    subject_name = subject_name.strip()
    subject_text = subject_text.strip()

    if not subject_name or not subject_text:
        print(f"Skipping malformed subject: {repr(subject_name)}")
        continue
    print(f"Parsing subject: {repr(subject_name)}")
    print(f"Text start: {repr(subject_text[:100])}")

    subject_text = re.split(r'^###\s+', subject_text, flags=re.MULTILINE)[0].strip()
    
    steps = {}
    state_type_to_step = {}
    
    lines = subject_text.strip().splitlines()
    current_step = None


    i = 0
    while i < len(lines):
        line = lines[i].strip()

        
        match = re.match(r'^(\d+)\.\s*\*{0,2}(\w+)\*{0,2}:\s*(.*)', line)
        if match:
            num, state_type, label = match.groups()
            current_step = int(num)
            state_type_to_step[state_type] = current_step 

            if state_type == "GotoStep":
                target_step = int(label.strip().split()[0])
                steps[current_step] = {
                    "type": "GotoStep",
                    "label": "",
                    "choices": [],
                    "branches": [],
                    "goto": target_step
                }
            else:
                steps[current_step] = {
                    "type": state_type,
                    "label": label,
                    "choices": [],
                    "branches": [],
                    "goto": None
                }
                # If it's a SendState, look ahead to capture 'To' and 'Msg'
                if state_type == "SendState":
                    if i + 1 < len(lines) and lines[i + 1].strip().startswith("To:"):
                        steps[current_step]["To"] = lines[i + 1].strip().split(":", 1)[1].strip()
                        i += 1
                    if i + 1 < len(lines) and lines[i + 1].strip().startswith("Msg:"):
                        steps[current_step]["Msg"] = lines[i + 1].strip().split(":", 1)[1].strip()
                        i += 1

                # For StartState and DoState, check if next line(s) contain OutgoingLabel
                if state_type in {"StartState", "DoState"}:
                    # Look ahead for OutgoingLabel line(s)
                    if i + 1 < len(lines) and lines[i + 1].strip().startswith("OutgoingLabel:"):
                        steps[current_step]["OutgoingLabel"] = lines[i + 1].strip().split(":", 1)[1].strip()
                        i += 1
                    # If no OutgoingLabel, look for Description and use that as OutgoingLabel
                    elif i + 1 < len(lines) and lines[i + 1].strip().startswith("Description:"):
                        steps[current_step]["OutgoingLabel"] = lines[i + 1].strip().split(":", 1)[1].strip()
                        i += 1
                i += 1
                continue

        # Parse ReceiveState choices
        
        elif re.match(r'^\s*-?\s*From:', line) and current_step is not None:
            
            match_from = re.match(r'^\s*-?\s*From:\s*(.+)', line)
            if match_from:
                from_actor = match_from.group(1).strip()
            
                msg = ""
                next_step = None

            if i + 1 < len(lines) and lines[i + 1].strip().startswith('Msg:'):
                msg = lines[i + 1].strip().split(':', 1)[1].strip()
                i += 1
            if i + 1 < len(lines) and lines[i + 1].strip().startswith('Next:'):
                next_val = lines[i + 1].strip().split(':', 1)[1].strip()
                if next_val.isdigit():
                    next_step = int(next_val)
                else:
                    # Resolve by state_type_to_step dictionary
                    next_step = state_type_to_step.get(next_val)
                    if next_step is None:
                        print(f"Warning: Could not resolve 'Next' target '{next_val}' for step {current_step}")
                steps[current_step]['next'] = next_step
                i += 1

            steps[current_step]["choices"].append({
                "from": from_actor,
                "msg": msg,
                "next": next_step
            })
            print(f"Parsed ReceiveState choice at step {current_step}: from={from_actor}, msg={msg}, next={next_step}")
        # Parse DoState branches
        elif line.startswith('- Step:') and current_step is not None:
            
            
            step = line.split(':', 1)[1].strip()
            if step.isdigit():
                step = int(step)
                desc = ""
                if i + 1 < len(lines) and lines[i + 1].strip().startswith('Description:'):
                    desc = lines[i + 1].strip().split(':', 1)[1].strip()
                    i += 1
                steps[current_step]["branches"].append({
                    "step": step,
                    "desc": desc
            })
            else:
                print(f"Skipping invalid Step line: {line}")
                # Just skip lines where step is not an integer

        # Goto inside other step types
        elif 'GotoStep' in line and current_step is not None:
            
            match = re.match(r'\*{0,2}GotoStep\*{0,2}:\s*(\d+)', line)
            if match:
                steps[current_step]["goto"] = int(match.group(1))

        i += 1

    # Add dummy nodes for unresolved goto targets
    goto_targets = {info["goto"] for info in steps.values() if info["goto"] is not None}
    for tgt in goto_targets:
        if tgt not in steps:
            steps[tgt] = {
                "type": "Unknown",
                "label": f"Step {tgt} \\nundefined",
                "choices": [],
                "branches": [],
                "goto": None
            }
      # Add this block here to ensure all `next:` targets (including EndStates) exist
    for num, info in steps.items():
        for choice in info.get("choices", []):
            target = choice.get("next")
            if target is not None and target not in steps:
                print(f"Warning: Step {target} missing in steps — adding as unknown.")
                steps[target] = {
                    "type": "Unknown",
                    "label": f"Step {target} \\nmissing",
                    "choices": [],
                    "branches": [],
                    "goto": None
                }
    # debug print to check parsed steps before drawing graph
    print("Parsed steps:")
    for k, v in steps.items():
        print(k, v)

    # === Draw state nodes ===
    for num, info in steps.items():
        if info["type"] == "GotoStep":
            continue  # skip drawing node

        if info["type"] == "Unknown" and info["label"].endswith("undefined"):
            continue

        style = state_styles.get(info["type"], {"shape": "rectangle", "color": "lightgrey"})
        dot.node(str(num), label=info["label"], shape=style["shape"], style="filled", fillcolor=style["color"])

    # === Draw transitions ===
    all_step_nums = sorted(steps.keys())

    for idx, num in enumerate(all_step_nums):
        info = steps[num]

        if info["type"] == "GotoStep":
            continue  # Only draw the goto edge, skip all else

        if info["type"] == "EndState":
            continue

        if info["type"] == "ReceiveState":
            if info["choices"]:
                for choice in info["choices"]:
                    
                    label = f"From: {choice['from']}\\nMsg: {choice['msg']}"
                    dot.edge(str(num), str(choice["next"]), label=label)
            elif "next" in info and info["next"] is not None:
                # Default path (no choices, just next)
                dot.edge(str(num), str(info["next"]))

        elif info["type"] == "SendState":
            label_parts = []
            if "To" in info:
                label_parts.append(f"To:{info['To']}")
            if "Msg" in info:
                label_parts.append(f"Msg:{info['Msg']}")
            label = "\\n".join(label_parts)
            
            next_step = info.get("next")
            if not next_step:
                idx = all_step_nums.index(num)
                next_step = all_step_nums[idx + 1] if idx + 1 < len(all_step_nums) else None

            
            if next_step:
                print(f"Drawing edge from {num} to {next_step} with label {label}")
                dot.edge(str(num), str(next_step), label=label)

        elif info["type"] == "StartState":
            next_step = all_step_nums[idx + 1] if idx + 1 < len(all_step_nums) else None
            if next_step:
                label = info.get("OutgoingLabel", info["label"])
                if label and subject_name.strip().lower().startswith("customer"):
                    dot.edge(str(num), str(next_step), label=label)
                else:
                    dot.edge(str(num), str(next_step))  # No label for system side
                

        elif info["type"] == "DoState":
            if info["branches"]:
                for branch in info["branches"]:
                    target_step = branch["step"]
                    target_info = steps.get(target_step)
                    if target_info and target_info["type"] == "GotoStep" and target_info.get("goto"):
                        real_target = target_info["goto"]
                        dot.edge(str(num), str(real_target), label=branch["desc"])
                    else:
                        dot.edge(str(num), str(target_step), label=branch["desc"])
                continue
            else:
                # No branches — fall back to sequential next step using description as label
                next_step = all_step_nums[idx + 1] if idx + 1 < len(all_step_nums) else None
                if next_step:
                    label = info.get("OutgoingLabel", info["label"])
                    dot.edge(str(num), str(next_step), label=label)


        if info["goto"]:
            dot.edge(str(num), str(info["goto"]))
            continue

        next_step = all_step_nums[idx + 1] if idx + 1 < len(all_step_nums) else None
        
        
        # Use explicit 'next' if present (from Next: EndState or Next: step_num)
        next_step = info.get("next")
        if not next_step:
            idx = all_step_nums.index(num)
            next_step = all_step_nums[idx + 1] if idx + 1 < len(all_step_nums) else None
        has_outgoing = (
        bool(info["choices"]) or
        bool(info["branches"]) or
        bool(info["goto"]) or
        info["type"] in {"SendState", "DoState", "StartState"}
        )

        if next_step and not has_outgoing and info["type"] not in {"ReceiveState"}:
            dot.edge(str(num), str(next_step), label="(default)")

    
    safe_subject_name = safe_filename(subject_name)
    dot.render(f"/home/s/smanan/SBD_{safe_subject_name}", cleanup=True)
    

Total blocks: 3
Blocks: ['', 'Customer', '\n1. StartState: Decide to place order  \n   Description: Customer initiates order process\n\n2. SendState: Send Order to E-commerce Website  \n   To: E-commerce Website  \n   Msg: Order  \n   Next: 3\n\n3. ReceiveState: Receive response from E-commerce Website  \n   Choices:\n   - From: E-commerce Website  \n     Msg: Payment Request  \n     Next: 4  \n   - From: E-commerce Website  \n     Msg: Order Decline  \n     Next: 6  \n\n4. SendState: Send Payment to Payment Gateway  \n   To: Payment Gateway  \n   Msg: Payment  \n   Next: 5\n\n5. ReceiveState: Receive Payment Confirmation from Payment Gateway via E-commerce Website  \n   From: E-commerce Website  \n   Msg: Order Confirmation  \n   Next: 7\n\n6. DoState: Decide upon further action after order decline  \n   Branches:\n   - Step: 8  \n     Description: Cancel order  \n   - Step: 10  \n     Description: Choose another supplier and try again\n\n7. EndState: Order confirmed (end of this path

In [1]:
from openai import OpenAI  ##########Updated Prompt  VERSION - ###################

# Setup
api_key = ""
base_url = "https://gpt.uni-muenster.de/v1"
model = "Llama-3.3-70B"  # model used

client = OpenAI(api_key=api_key, base_url=base_url)

# Define scenario
scenario = """
A customer places an order on an e-commerce website. 
The system checks inventory. 
If the item is in stock, customer is informed to pay for the product.
Customer pays for the order.
The payment gateway confirms the payment transaction. 
Finally, the system sends order confirmation email to the customer.
But if the item is not in stock, order is declined.
Customer is sent order declined message.
Customer then decides upon further action either the customer decides to cancel order and it ends
or customer chooses another e-commerce supplier.
"""

# Create the prompt
prompt = f"""

You are an expert in Subject-Oriented Business Process Modeling using the Parallel Activity Specification Schema (PASS).

Analyze the following scenario step-by-step and produce both the Subject Interaction Diagram (SID) and Subject Behavior Diagrams (SBDs).

\"\"\"{scenario}\"\"\"

### Step 1: Identify Subjects

List all autonomous subjects (actors or systems) involved in the scenario.

### Step 2: Identify Messages (Noun Form Only)

List key messages exchanged between subjects using **noun phrases only** (e.g., “Order”, “Inventory Status” etc.).

## Step 3: Subject Interaction Diagram (SID)

Write the SID as a **numbered list** describing subject-to-subject message flows.

Format:
'[Subject A] -> [Subject B]: [Message]'

### Step 4: Think Step-by-Step (REASONING PHASE)

- Break down the subject's journey into clear events.
- Identify each type of PASS state involved.
- Consider conditional branches (e.g., item in stock vs out of stock).
- Match each message in the SID to the appropriate state.
- Ensure logical transitions and valid branches or GotoSteps.
-When the subject receives a Declination Message (e.g. item out of stock), the flow must proceed to a 
 DoState: Decide upon further action, where the subject either cancels the order or chooses another supplier. 
 This ensures proper branching and no immediate sending of cancellation message without decision.
 
-When the subject receives e.g. a payment request, they must first send payment, then receive payment confirmation, 
and then transition to the final state (e.g., Order Confirmed).

-The Order Declination branch must go to a DoState (decision) and must not pass through payment states.

-Never allow decision branches to lead to states meant for a different outcome (e.g., do not send to "Receive Payment Confirmation" if no payment was sent).

**Please write out your reasoning step-by-step before generating the Subject Behavior Diagram (SBD).**

### Step 5: Subject Behavior Diagram (SBD)

For each subject from Step 1, describe its internal behavior using PASS state types:

- **StartState**, **DoState**, **SendState**, **ReceiveState**, **EndState**

**Rules**:
- 'SendState' :- include 'To:' and 'Msg:'
- 'ReceiveState' :- include 'From:' and 'Msg:'
- Use 'Branches:' for decisions
  Always increment step numbers correctly. Double-check that Branches: refer to valid steps (e.g., "Step: 10", not "Step: 9" if Step 9 is an EndState).
- Use GotoStep: to return to earlier steps (e.g., sending new order again)
-For retry paths (like choosing another supplier), always use GotoStep: instead of duplicating behaviour"
-Ensure branches reference the correct step numbers

- In decision branches:
  - Each `Step:` must point to a valid numbered step that **actually exists**.
  - If retrying earlier steps, use `GotoStep:` and **refer to original step number** (e.g., GotoStep: 2).
- Do not invent new EndStates or GotoSteps that aren't clearly part of a branch.

Example Branch:

6. DoState: Decide next action
   Branches:
   - Step: 7
     Description: Cancel order
   - Step: 9
     Description: Try another supplier

7. SendState: Send Cancellation
8. EndState: Order Cancelled
9. GotoStep: 2  # Retry order

**Important Note on Step Numbering and 'Next:' References:**

- Each step number must be sequential unless explicitly branching or looping back.
- The 'Next:' field in each state should refer to the **very next logical step in the process**, not skip decision points or intermediate steps.
- If a decision is required next, 'Next:' should point to the corresponding DoState step, not jump directly to an EndState or unrelated step.
- Only use 'Next: EndState' when the flow truly terminates at that step. And in 'Next: EndState', put EndState
  number like 'Next: 8) if 8.EndState
- Avoid skipping over intermediate steps such as decisions or sends; maintain logical process flow.

- After receiving confirmations or important messages, flow should proceed to the next logical decision or processing step, not directly to an EndState unless the process actually terminates.

****Important Notes on EndStates:
Use separate EndStates for different outcomes:
e.g., one for “Successfully Completed”, another for “Cancelled”.
Do not route cancellation, decline, or failed attempts to an EndState meant for successful completions.
If a common EndState is used for multiple paths, name it generically, e.g., “Process Completed” or “End of Process”.
Ensure Next: fields route only logically to the correct EndState matching the path.

Model each subject one by one.

### Step 6: Check Logic and Flow

Ensure:
- Each decision branch leads to an `EndState` or valid `GotoStep:`
- No duplicate messages
- Each ReceiveState handles only mutually exclusive messages

### Example Scenario (Healthcare):
A patient books an appointment using a healthcare app.  
The app sends the appointment request to the hospital system.  
The hospital system checks the doctor’s availability.  
If the doctor is available, it confirms the appointment.  
The app then sends a confirmation message to the patient.
If the doctor is not available, hospital system declines the appointment.
The patient is sent decline message.
The patient then decides upon further action either the patient cancels request and all ends
or the patient chooses another hospital.

### SBD:
####Patient:
1. StartState: Decide to make appointment  
   Description: Patient initiates appointment process

2. SendState: Send Appointment Request to Healthcare App  
   To: Healthcare App  
   Msg: Appointment Request

3. ReceiveState: Receive Reply from Healthcare App  
   Choices:
   - From: Healthcare App  
     Msg: Appointment Confirmation  
     Next: 4  
   - From: Healthcare App  
     Msg: Appointment Declination  
     Next: 5  

4. EndState: Appointment booked

5. DoState: Decide upon further action  
   Branches:
   - Step: 6  
     Description: Cancel appointment request  
   - Step: 8  
     Description: Choose another hospital and try again

6. DoState: Cancel appointment request  
   Description: Cancel

7. EndState: Appointment request cancelled (end of this path)
8. GotoStep: 2  # Retry appointment loop

(Similar steps would follow for Healthcare App and Hospital System…)
Now, do the same for the scenario provided above. Output the **SID first**, followed by **SBDs for each subject**.
Model all subjects from Step 1. Do not skip the last subject. Ensure each subject has its full SBD.
Repeat Steps 4–5 for **every subject** listed in Step 1. 
Do not skip subjects 
Ensure each has a complete behavior diagram from StartState to EndState.
"""

# Make the API call
completion = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model=model,
    temperature=0.5    
)

# Output
print(completion.choices[0].message.content)

### Step 1: Identify Subjects

The autonomous subjects (actors or systems) involved in the scenario are:
1. Customer
2. E-commerce Website (System)
3. Payment Gateway

### Step 2: Identify Messages

Key messages exchanged between subjects are:
- Order
- Inventory Status
- Payment Request
- Payment Confirmation
- Order Confirmation
- Order Declination
- Cancellation

### Step 3: Subject Interaction Diagram (SID)

Here is the SID as a numbered list describing subject-to-subject message flows:

1. Customer -> E-commerce Website: Order
2. E-commerce Website -> E-commerce Website: Inventory Status (internal check)
3. E-commerce Website -> Customer: Payment Request (if item is in stock)
4. Customer -> Payment Gateway: Payment
5. Payment Gateway -> Customer: Payment Confirmation
6. Customer -> E-commerce Website: Payment Confirmation
7. E-commerce Website -> Customer: Order Confirmation
8. E-commerce Website -> Customer: Order Declination (if item is not in stock)
9. Customer -> Customer: Dec